In [1]:
import pandas as pd
import re

In [2]:
sv_df = pd.read_csv('/s/project/mll/sergey/MLL_data/vale.svs.filtered.txt', sep='\t')

/scratch/tmp/l_vilov/ipykernel_886130/810917226.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sv_df = pd.read_csv('/s/project/mll/sergey/MLL_data/vale.svs.filtered.txt', sep='\t')


In [3]:
sv_df = sv_df[~(sv_df.gnomad>5e-4)] #only structural variants with low gnomAD AF

sv_df = sv_df[~sv_df.to_gene.isna()|~sv_df.from_gene.isna()] #only structural variants intersecting with genes

In [4]:
def make_gene_list(x):
    gene_list = str(x.to_gene) + ',' + str(x.from_gene)
    gene_list = re.sub('\([-\+]\)','', gene_list)   
    gene_list = re.sub(',*nan,*','', gene_list)   
    if ',' in gene_list:
        gene_list = gene_list.split(',')
    return gene_list

sv_df['geneHGNC'] = sv_df.apply(make_gene_list, axis=1)

In [5]:
sv_df = sv_df[['array_id','svtype','geneHGNC']].explode('geneHGNC')

In [6]:
matching_genes = pd.read_csv('/s/project/mll/sergey/effect_prediction/promoter_mutations/ensemble_to_HGNC_GRCh38.tsv.gz', sep='\t', 
                      header=None, names=['geneName', 'geneHGNC'], usecols=[0,1], skiprows=1)

matching_genes = matching_genes[~matching_genes.geneHGNC.isna()]

In [7]:
sv_df = sv_df.merge(matching_genes, how='left')

In [8]:
sv_df

,array_id,svtype,geneHGNC,geneName
0,MLL_11389,DEL,TNFRSF8,ENSG00000120949
1,MLL_11389,DEL,TNFRSF8,ENSG00000120949
2,MLL_11389,INV,PTGFR,ENSG00000122420
3,MLL_11389,INV,PTGFR,ENSG00000122420
4,MLL_11389,BND,MECOM,ENSG00000085276
...,...,...,...,...
3202866,MLL_11293,INV,VCX3B,ENSG00000205642
3202867,MLL_11293,BND,IL1RAPL1,ENSG00000169306
3202868,MLL_11293,DEL,TM4SF2,NaN
3202869,MLL_11293,DEL,TM4SF2,NaN


In [9]:
sv_df.to_csv('/s/project/mll/sergey/MLL_data/processed/vale.svs.filtered.tsv.gz', sep='\t', index=None)